In [1]:
!pip3 install PyGithub requests nb_black
!pip3 install ipython --upgrade

     |████████████████████████████████| 289 kB 22.1 MB/s eta 0:00:01
  Created wheel for wrapt: filename=wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl size=69764 sha256=049d2f5a9b8db3a7ffb2a62139af8880eea2afc094d24e9419aeb5f306f50730
  Stored in directory: /home/cdsw/.cache/pip/wheels/32/42/7f/23cae9ff6ef66798d00dc5d659088e57dbba01566f6c60db63
Successfully built wrapt
You should consider upgrading via the '/usr/local/bin/python3.6 -m pip install --upgrade pip' command.


In [1]:
import os
import pickle
import datetime
import pandas as pd
from github import Github

%load_ext lab_black

In [2]:

AMP_NAMES = [
    "Churn_Prediction",
    "Image_Analysis",
    "Anomaly_Detection",
    "NeuralQA",
    "Structural_Time_Series",
    "SpaCy_Entity_Extraction",
    "Explainability_LIME_SHAP",
    "Question_Answering",
    "Active_Learning",
    "MLFlow_Tracking",
]

AMP_REPOS = ["cloudera/CML_AMP_" + amp for amp in AMP_NAMES]

### Pull last 14 days of data

In [3]:
def get_usage_last_14_days(gh_token, amp_repos):
    """
    Provided a GH API token and a list of qualified Github repository names that the
    API token has access to, this function pulls usage metrics (clones & views) for
    each repository for the last 14 days, and returns as a Pandas DataFrame. It also
    collects aggregate metrics on the sources sites driving traffic to each repo
    over the last 14 days.

    Args:
        gh_token (str)
        amp_repos List[str]

    Returns:
        amp_tracking_df (pd.DataFrame)
        amp_referring_df (pd_DataFrame)

    """

    gh = Github(gh_token)

    activity_dfs = []
    referring_dfs = []

    for repo in amp_repos:
        gh_repo = gh.get_repo(repo)

        # gather referring sites as DF
        refs = gh_repo.get_top_referrers()
        ref_data = []
        for ref in refs:
            data = {
                "referrer": ref.referrer,
                "refs_unique": ref.uniques,
                "refs_total": ref.count,
            }
            ref_data.append(data)
        ref_df = pd.DataFrame(ref_data)
        ref_df["repo"] = repo[17:]
        referring_dfs.append(ref_df)

        # gather view activity as DF
        views = gh_repo.get_views_traffic(per="day")
        view_data = []
        for view in views["views"]:
            data = {
                "timestamp": view.timestamp,
                "views_unique": view.uniques,
                "views_total": view.count,
            }
            view_data.append(data)
        view_df = pd.DataFrame(view_data).set_index("timestamp")
        idx = pd.date_range(
            end=pd.to_datetime("today").date().strftime("%m-%d-%Y"),
            start=(
                pd.to_datetime("today").date() - datetime.timedelta(days=14)
            ).strftime("%m-%d-%Y"),
        )
        view_df = view_df.reindex(idx, fill_value=0)

        # gather clone activity as DF
        clones = gh_repo.get_clones_traffic(per="day")
        clone_data = []
        for clone in clones["clones"]:
            data = {
                "timestamp": clone.timestamp,
                "clones_unique": clone.uniques,
                "clones_total": clone.count,
            }
            clone_data.append(data)

        clone_df = pd.DataFrame(clone_data).set_index("timestamp")
        clone_df = clone_df.reindex(idx, fill_value=0)

        # combine DFs
        activity_df = pd.concat([clone_df, view_df], axis=1)
        activity_df["repo"] = repo[17:]

        activity_dfs.append(activity_df)

    amp_tracking_df = pd.concat(activity_dfs)
    amp_referring_df = pd.concat(referring_dfs).reset_index(drop=True)

    return amp_tracking_df, amp_referring_df

### Update Live Data

Every morning at 8 a.m. EST, a Job will run that:
1. Gets the latest 14 day usage stats (both tracking and referring)
2. Pulls the 2nd to last day stats for the tracking metrics(basically the final full count of yesterdays stats) and save to daily_archive folder
3. Loads yesterday's production tracking_df archive pkl file and appends latest days stats to it
4. Saves a pkl of the new cumulative DF and new referring df (in archive folder)
5. Deletes the existing tables
6. Recreates the tables (this ensures we always have an IDENTICAL backup as a pkl DF)

Note that the datetime module is ahead of the Github timezone by ~5 hours, so by running this every morning at 8 a.m. and taking the second to last day, we'll always capture the full prior day

In [6]:
# 1. Get latest 14 day usage - both tracking and referring DFs
amp_tracking_df, amp_referring_df = get_usage_last_14_days(
    gh_token=TOKEN, amp_repos=AMP_REPOS
)

# 2. Pull just the second to last days stats from tracking DF (basically the final full count of yesterdays stats) and save to daily_archive
completed_day = amp_tracking_df.index[-2]
daily_tracking_df = amp_tracking_df.loc[completed_day]

today_str = datetime.datetime.today().strftime("%m-%d-%Y")
yesterday_str = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime(
    "%m-%d-%Y"
)
os.makedirs(f"data/daily_archive/{today_str}")

daily_tracking_df.to_pickle(
    f"data/daily_archive/{today_str}/daily_tracking_{today_str}.pkl"
)

amp_referring_df.to_pickle(
    f"data/daily_archive/{today_str}/daily_referring_last14_{today_str}.pkl"
)

In [9]:
# 3. Load yesterday's production tracking_df and append new daily_tracking_df
old_prod_tracking = pd.read_pickle(
    f"data/prod_archive/{yesterday_str}/cumulative_tracking_{yesterday_str}.pkl"
)
new_prod_tracking = pd.concat([old_prod_tracking, daily_tracking_df])

os.makedirs(f"data/prod_archive/{today_str}")
new_prod_tracking.to_pickle(
    f"data/prod_archive/{today_str}/cumulative_tracking_{today_str}.pkl"
)

In [14]:
# 4. Delete existing SQLite tables
conn = create_connection(f"{os.getcwd()}/db/pythonsqlite.db")

delete_table("amp_tracking", conn)
delete_table("amp_referring", conn)

# 5. Create new tables to refresh data

create_table_from_df("amp_tracking", conn, new_prod_tracking)
create_table_from_df("amp_referring", conn, amp_referring_df)

### SQLite DB Utils

In [13]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """
    Create a database connection to the SQLite database specified by db_file

    Args:
        database file

    Returns:
        Connection object
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_table_from_df(table_name, conn, df):
    """
    Create a SQLite table given the name, connection, and
    pandas dataframe

    """
    df.to_sql(name=table_name, con=conn)


def delete_table(table_name, conn):
    """
    Delete a table in a SQLite DB given the connection and table name

    """
    cur = conn.cursor()

    qry = f"DROP TABLE {table_name}"
    cur.execute(qry)


def select_all_from_table(table_name, conn):
    """
    Query a table in a SQLite DB given the connection and table name
    for all records and return as pandas dataframe

    """
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    return df

In [37]:
conn = create_connection(f"{os.getcwd()}/db/pythonsqlite.db")

In [41]:
create_table_from_df("amp_tracking", conn, amp_tracking_df)
create_table_from_df("amp_referring", conn, amp_referring_df)

In [15]:
cur = conn.cursor()

In [18]:
cur.execute("PRAGMA table_info(amp_tracking)")

In [19]:
cur.fetchall()

[(0, 'index', 'TIMESTAMP', 0, None, 0),
 (1, 'clones_unique', 'INTEGER', 0, None, 0),
 (2, 'clones_total', 'INTEGER', 0, None, 0),
 (3, 'views_unique', 'INTEGER', 0, None, 0),
 (4, 'views_total', 'INTEGER', 0, None, 0),
 (5, 'repo', 'TEXT', 0, None, 0)]